In [1]:
import fiona

In [2]:
# building shape file
shape_file = '/Users/sukryool.kang/Projects/Geospatial Project/Data_For AI Team/LA__Road_Ground_truth/Vector_LA1-32-33-35-36-37/LA1-32-33-35-36-37.shp'

#road shape file
shape_file = '/Users/sukryool.kang/data/GIS_data/LosAngeles_2017/Phase 1 Road Polygon/Phase 1 Road Polygon.shp'


In [3]:
gt_shape = fiona.open(shape_file)

for idx,feat in enumerate(gt_shape):
    break

In [4]:
print(feat)

{'type': 'Feature', 'id': '0', 'geometry': {'type': 'Polygon', 'coordinates': [[(379933.04825826833, 3762948.420827571), (379922.04814001144, 3762948.4001117176), (379922.0500745137, 3762949.01984001), (379933.04973792576, 3762948.894843835), (379933.04825826833, 3762948.420827571)]]}, 'properties': OrderedDict([('osm_id', '159705428'), ('code', 5114), ('fclass', 'secondary'), ('name', 'Melrose Avenue'), ('ref', None), ('oneway', 'B'), ('maxspeed', 0), ('layer', 0), ('bridge', 'F'), ('tunnel', 'F'), ('path', 'C:\\Sans\\AI Project\\Geofabrik data\\08.01.2019\\Road_Types\\fclass_secondary_buff.shp')])}


In [5]:
print(len(gt_shape))

2


In [6]:
print(gt_shape[0])

{'type': 'Feature', 'id': '0', 'geometry': {'type': 'Polygon', 'coordinates': [[(379933.04825826833, 3762948.420827571), (379922.04814001144, 3762948.4001117176), (379922.0500745137, 3762949.01984001), (379933.04973792576, 3762948.894843835), (379933.04825826833, 3762948.420827571)]]}, 'properties': OrderedDict([('osm_id', '159705428'), ('code', 5114), ('fclass', 'secondary'), ('name', 'Melrose Avenue'), ('ref', None), ('oneway', 'B'), ('maxspeed', 0), ('layer', 0), ('bridge', 'F'), ('tunnel', 'F'), ('path', 'C:\\Sans\\AI Project\\Geofabrik data\\08.01.2019\\Road_Types\\fclass_secondary_buff.shp')])}


In [7]:
print(gt_shape[1])

{'type': 'Feature', 'id': '1', 'geometry': {'type': 'Polygon', 'coordinates': [[(379930.62314613396, 3766970.758063728), (379924.0459770041, 3766970.758063728), (379889.1540947704, 3766970.8896071105), (379867.28500741365, 3766971.218465567), (379839.1182806152, 3766972.994301232), (379818.79482800403, 3766975.8882556492), (379790.8418592022, 3766980.656703269), (379775.7163259058, 3766982.7429837235), (379775.7362595693, 3766982.8904928337), (379769.4989453759, 3766983.7150869137), (379769.5214853849, 3766983.8751209774), (379744.1768442261, 3766987.2503153207), (379731.1211635034, 3766988.7466212977), (379720.20306274784, 3766989.9962834325), (379719.9778448725, 3766990.1024575736), (379698.070888626, 3766992.9560095407), (379675.24811174546, 3766996.047279032), (379672.88033085875, 3766996.573452562), (379672.55070988, 3766996.606351393), (379664.3135680671, 3766997.4284845493), (379623.50223361654, 3767002.1640463225), (379598.90362107096, 3767003.8741102964), (379598.6391422719, 3